In [ ]:
import json
import zlib
import base64

def decode(string):
    return json.loads(zlib.decompress(base64.b64decode(string[1:])).decode('UTF-8'))

def encode(obj):
    return '0' + base64.b64encode(zlib.compress(bytes(json.dumps(obj), 'UTF-8'))).decode('UTF-8')

def encodePr(obj):
    print(encode(obj))

def printEntities(ent):
    assert isinstance(ent, list)
    print(encode({'blueprint': {'entities': ent}}))


In [66]:
S='0eNrtWsuumzAQ/RevyRU2zyB125+oKpSAb67VxESOiRpF/HtNqNJcg59dtEjeRIKYMTPnMQPJHeyPPT4zQjmo7oA0Hb2A6tsdXMiB7o7jOX47Y1ABwvEJRIDuTuNRixvSYrZputOe0B3vGBgiQGiLf4IKDt8jgCknnOAp2uPgVtP+tMdMLHjGGffjO8pfA0Xg3F3EtR0dtx/j5cVbFoEbqBB8y8Q+LWG4mRagaIzBWXes9/hjdyUigLjqnRw5ZopUroTxXpx53sUPtsEUs8Ntg8Y0mq4fywFfE3qcpnTa9jIGg+MHw+1rgkQc5WLxMESzpJFr0uWKki4VSSeuSW9XlDSMFVmnjlkX8ZqyRoqsM5076ESdfk45XU75d9BafNeS512+E3bhtbkE04rN1yn/CZRHCcQ9nnfscY8V+CIu6Hp+7h1C4itmN/5B6GGKfb7VjwLX76w71YSKYKDirMeDi4dEn2sudNQQ1vSET8ejwx4YxlS+spgtFLsixTbSYhWwubddoxXQWSp1Ni/gUk0KR7KXgezq3iGRPbUl+wyrz4G2LlrIZC1oY6mkUno3+RVIRa5JYSeVraNUtkEqmolD0krm3RikQPCvGoc+mEotMPYeD1cgl8RsR4tFgW56+TM8Br0szKoSM3NbvTg1D8vuAJH3g8EK+J5aWMpiVRLfB4f8nxE+/k8Jb5iIYGFL/4UhTMN/ebVSAKnv2Bygdm3opTXUiSFS5kSFxDAbZJZUyXzHxkAVYy+TINn6u4KxvTq5hj6akiq578QUqDLDBOoRRrE1V9xsw9YWCt9hoQzTsaPxI2iNdemEdWmJdek7LQSsXZ0bIWusC+duouOC6VWC7aOV91umwBWzc0tcSfx9wdhNnHxDH03FFRT7zguBKzNMcgPC1i/zHX3D0hcQ9JwXkjjMhkbnV2G50E00WMqrlVgiz3kgYNlaTGwqLKFzN9BhDQ0enlhyIfHs94ELrYXzWuva6PZOutdHU3Ih9ezngQtzTKyf7x11v6hrEfvxv8Lq5W+IERBpXabfOEqYFltUFGVSJkk+DL8AjTbt2A=='
S='0eNqNkNFqwzAMRf9Fz24hSZtk/pVRip1qm8CRg6OUhuB/r5VC2dPYo8S95+pqAx8WnBKxgN2Ahsgz2M8NZvpmF3Qn64RggQRHMMBu1El14lgOQxw9sZOYIBsgvuEDbJUvBpCFhPCF24f1ysvoMRXBnyADU5yLN7LmK6+tjmcDK9i6PZ5Lzo0SDi/ByShDUgxXjz/uTgVQXF8UBNO/ukhyPE8xycFjEK0xxEX/0b0L1fmStdNutL9+ZuBeUvZD6r46dR911/VN3zRtzk9ga3UG'
X = decode(S)
X['blueprint']['entities']


[{'entity_number': 1,
  'name': 'constant-combinator',
  'position': {'x': 161.5, 'y': 26.5},
  'direction': 4,
  'control_behavior': {'filters': [{'signal': {'type': 'item',
      'name': 'transport-belt'},
     'count': 7,
     'index': 2}]}}]

In [67]:
class Blueprint:
    CONN_DECIDER_IN = '1'
    CONN_DECIDER_OUT = '2'
    CONN_CONSTANT = '1'

    def __init__(self, string=None):
        if string:
            self._root = decode(string)
        else:
            self._root = {'blueprint': {'entities': []}}

    @property
    def entities(self) -> list[dict]:
        return self._root['blueprint']['entities']

    def pr(self) -> None:
        string = encode(self._root)
        print(string)

    def _at(self, x, y) -> dict:
        found = [entity for entity in self.entities if abs(entity['position']['x'] - x) <= 0.5 and abs(entity['position']['y'] - y) <= 0.5]
        assert len(found) == 1
        return found[0]

    def create_constant(self, x, y):
        constant = {
            'entity_number': 1 + len(self.entities),
            'name': 'constant-combinator',
            'position': {'x': x + 0.5, 'y': y + 0.5},
            'direction': 2
        }
        self.entities.append(constant)

    # (x, y) is an upper part of decider
    def create_decider(self, x, y, f_value):
        decider = {
            'entity_number': 1 + len(self.entities),
            'name': 'decider-combinator',
            'position': {'x': x + 0.5, 'y': y + 1},
            'direction': 4,
            'control_behavior': {'decider_conditions': {'first_signal': {'type': 'virtual',
                'name': 'signal-F'},
                'constant': f_value,
                'comparator': '=',
                'output_signal': {'type': 'virtual', 'name': 'signal-everything'},
                'copy_count_from_input': True}}
        }
        self.entities.append(decider)

    def add_item(self, x, y, name, count):
        entity: dict = self._at(x + 0.5, y + 0.5)
        assert entity['name'] == 'constant-combinator'
        filters: list = entity.setdefault('control_behavior', {}).setdefault('filters', [])
        index = 1 + len(filters)
        assert index <= 20

        item = {
            'signal': {
                'type': 'item',
                'name': name
            },
            'count': count,
            'index': index
        }
        filters.append(item)

    def add_connection(self, x, y, xy_canal, s, t, st_canal, color):
        entity_1: dict = self._at(x + 0.5, y + 0.5)
        entity_2: dict = self._at(s + 0.5, t + 0.5)
        connection1: list = entity_1.setdefault('connections', {}).setdefault('xy_canal', {}).setdefault(color, [])
        connection2: list = entity_2.setdefault('connections', {}).setdefault('st_canal', {}).setdefault(color, [])

        connection1.append({'entity_id': entity_2['entity_number'], 'circuit_id': int(st_canal)})
        connection2.append({'entity_id': entity_1['entity_number'], 'circuit_id': int(xy_canal)})
